In [9]:
import random
from requests import get, Session, request
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import sqlite3
from time import time, localtime, sleep, strftime, mktime
from datetime import datetime, timedelta
import json

session = Session()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
         'Referer': 'https://www.yna.co.kr/'}

In [ ]:
def robotParser(domain):
    url = urlunparse(urlparse(domain)[:2] + ('',)*4)
    url += '/robots.txt'    
    pathEnable = dict()
    resp = get(url)
    if resp.status_code == 200:
        agent = None
        for line in resp.text.splitlines():
            k, *v = line.split(':')
            k = k.strip()
            v = ':'.join(v).strip()
            if k.lower() == 'user-agent':
                agent = v
                if v not in pathEnable:
                    pathEnable[v] = dict()
            else:
                if k.lower() == 'allow':
                    pathEnable[agent][v] = True
                else:
                    pathEnable[agent][v] = False
    else:
        pathEnable['*'] = True
    return pathEnable

In [ ]:
def canFetch(pathEnable, path):
    agent = '*'
    path = urlparse(path).path
    
    if agent in pathEnable:
        if path in pathEnable[agent]:
            return pathEnable[agent][path]
        else:
            if path == '/':
                return True
            else:
                return canFetch(pathEnable,
                                '/'.join(path.split('/')[:-1]))
    else:
        return True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cd '/content/drive/MyDrive/Colab Notebooks'

In [12]:
# Initial parameters
# url = 'https://www.yna.co.kr/search/index'
today = datetime.today()
theday = today
today_string = datetime.strftime(today, '%Y%m%d')

query = dict()

categories = {'정치': '01', '경제': '02', '사회': '05', '금융증권산업': '03%2C04', '사건사고': '06', '문화': '07', '생활건강': '08', 'IT과학' : '09', '북한': '10', '국제': '11', '스포츠': '12', '연예': '0712'}

In [17]:
# crawl to db
con = sqlite3.connect('news_Crawl.db')
cur = con.cursor()

In [18]:
cur.execute('''
    SELECT id FROM visited WHERE bool = 0 LIMIT 100;
''')
ids = cur.fetchall()
ids, len(ids)

([], 0)

In [ ]:
# contents_id = 'AKR20221205120300009'
# page_url = 'https://www.yna.co.kr/view/{}'.format(contents_id)
# page_resp = get(page_url, headers=headers)
# dom = BeautifulSoup(page_resp.text, 'html.parser')
# category = dom.select_one('meta[property="article:section"]')['content']
# title = dom.select_one('h1.tit')

# content = dom.select('article.story-news.article p')[2:-2]
# #### 질문? 불필요한 tag 찾아서 정리하기

# [ar.string for ar in content if ar.string is not None]
# url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=1y&from=20211206&to=20221206&ctype=A&page_size=10&channel=basic_kr'.format(page_no)
# resp = get(url, headers=headers)
# ids = json.loads(resp.text)['KR_ARTICLE']['result']
# type(ids[0]['CONTENTS_ID'])

# params = {
#     'query': '%ED%96%88%EB%8B%A4',
#     'period': 'diy',
#     'page_no': page_no,
#     'ctype': 'A',
#     'from': today_string,
#     'to': today_string,
#     'page_size': 30,
#     'channel': 'basic_kr'
# }
# theday = today

In [14]:
while True:
    theday_string = datetime.strftime(theday, '%Y%m%d')
    # one day len(new) ~ 500, 12 categories * 30 ~ 480
    for page_no in range(1,14):
        url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=diy&from={}&to={}&ctype=A&page_size=30&channel=basic_kr'.format(page_no, theday, theday)
        resp = get(url, headers=headers)
        if resp.status_code != 200:
            break

        ids = json.loads(resp.text)['KR_ARTICLE']['result']
        for articleId in ids:
            idd = articleId['CONTENTS_ID']
            cur.execute('''
            INSERT OR IGNORE INTO visited (id) VALUES (:idd);
            ''', [idd])


    print(theday_string)
    cur.fetchall()
    # sleep(5*random.random())
    theday -= timedelta(days=1)

1 AKR20221207120100063
2 AKR20221207106600001
3 AKR20221207101500054
4 AKR20221207096800051
5 AKR20221207082800076
6 AKR20221207071351530
7 AKR20221207071000017
8 AKR20221207062200054
9 AKR20221207060100056
10 AKR20221207048300001
11 AKR20221207041400007
12 AKR20221207028900002
13 AKR20221206160500007
1 AKR20221207115600004
2 AKR20221207106600001
3 AKR20221207103800064
4 AKR20221207096600051
5 AKR20221207058100017
6 AKR20221207041751004
7 AKR20221207072900017
8 AKR20221207067000004
9 AKR20221207060100056
10 AKR20221207048300001


KeyboardInterrupt: 

In [19]:
# save article ids
cur.fetchall()
con.commit()
con.close()

In [ ]:
# get each pages
cur.execute('''
    SELECT id FROM visited WHERE bool = 0 LIMIT 100
''')
news_list = cur.fetchone()
print(len(news_list))
for article_id in news_list:
    page_url = 'https://www.yna.co.kr/view/{}'.format(article_id)
    page_resp = get(page_url, headers=headers)
    dom = BeautifulSoup(page_resp.text, 'html.parser')

    title = dom.select_one('h1.tit').string
    content = dom.select('article.story-news.article p')[2:-2]
    content = [ar.string for ar in content if ar.string is not None]
    content = ''.join(content)
    dateNews = article_id[4:14]
    category = dom.select_one('meta[property="article:section"]')['content']

    cur.execute('''
        INSERT INTO news
        (id, URL, title, content, dateNews, category)
        VALUES (:id, :URL, :title, :content, :dateNews, :category);
        ''', (article_id, page_url, title, content, dateNews, category))


cur.fetchall()
con.commit()
con.close()